# signal analysis notebook(Linux)
このノートブックでは、実際に筆者が書いた関数を呼び出し、それに基づいて信号の可視化を解析・解説しています。ソースコードは同じレポジトリ（src/utils.py）などを参照してください。
このファイルは、Linux向けに動作テストを実施しています。Windows環境では、別ファイルを参照してください。

## 環境設定  
下のセルを実行し、必要なライブラリをすべてインストールしましょう。GPUを使用しますので、きちんと認識されているか、Pythonで使用できる状態になっているか確認してください。  
なお、torchのインストールに関しては、筆者が動作確認を行った環境はやや古いためバージョンを指定する方法はうまく行きません。公式の対応に任せていますが、環境によっては別途での対応が求められるかもしれません。

In [ ]:
!pip install -r requirements.txt
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

## メタデータ表示
生のファイルの中にどのような情報があるのかを出力します。特に、これらはメタデータを表示するスクリプトとして利用してください。実際にデータが取得された年度において処理が異なることに留意してください。

In [ ]:
from src import analyze_mat_file,analyze_mat_file_h5py
file_path_exp = "/home/smatsubara/documents/airlift/data/experiments/rawsignal/P20240726-1600.mat"
file_path_sim = "/home/smatsubara/documents/airlift/data/simulation/rawsignal/solid_liquid2.mat"
analyze_mat_file(file_path_exp)
analyze_mat_file_h5py(file_path_sim)

## ファイル形式変換

実機データもシミュレーション生成のファイルも`.mat`形式で保存されていますが、これらはサンプリングレートや単位が異なるほか、実機データでは15000回の測定が連続的に記録されており、境目を表示することが困難です。  
ゆえに、処理の統一化を目的としてすべてを`.npz`形式に変換します。これによって、一つのスクリプトで統一的に信号波形を処理することができるようになります。また、機械学習用のデータセットを作成しやすくなるという狙いがあります。  
以下のセルは、実機データを変換するスクリプトです。   
### 実機データ
命名規則:
 - 2024: dates and time  
    e.g. P20241007-1013.mat
 - 2023: experimental settings(requires details)  
    e.g. s0_g2_l2_t1.mat

これらに対し、`.npz`形式に変換したものを`*_processed.npz`として新しい名前を与え保存しています。
なお、`signal_key="TDX1"`と書いてありますが、これはTDX1のみを処理しているのではなく、これらをベースにトリガを検出し、その時刻を基準点としてすべてのチャンネルの信号波形を並び変えているということに注意してください。



In [6]:
from src import mat2npz_exp


file_path = "/home/smatsubara/documents/airlift/data/experiments/rawsignal/P20241007-1013.mat"
output_dir = "/home/smatsubara/documents/airlift/data/experiments/processed"  # Change to your desired directory


# 関数を呼び出して実行
mat2npz_exp(
    file_path=file_path,
    output_dir=output_dir,
    start_time=0.0,
    duration=5.0,
    amplitude_threshold=2,
    window_width=0.1e-3,
    signal_key="TDX1"
)

Loading data...
Loading successful
Using device: cuda
Number of detected triggers: (14988,)
arranged_pulses_tdx1.shape: (14988, 5208)
arranged_pulses_tdx1_enlarged.shape: (14988, 5208)
arranged_pulses.shape: (14988, 5208, 4)
['__header__', '__version__', '__globals__', 'Tstart', 'Tinterval', 'ExtraSamples', 'RequestedLength', 'Length', 'Version', 'TDX1', 'TDX2', 'TDX3', 'TDX1_enlarged']
signal points: (5208,)
Processed data and metadata saved to: /home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz


'/home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz'

### シミュレーションデータ  
次に、シミュレーションデータを変換します。シミュレーションにおいては、計算時間の制約で複数パルス分のデータを用意することはできません。さらに、ソフトの設定によりkgridの中にdtは具体的な値としては保存されません。  
それゆえ、`config.json`をもとに復元する必要があります。そのため、`.json`ファイルも読み込んで同時に処理します。そこに、メタデータも含ませてよりリッチな情報を提供します。  
また`.npz`のファイルには１パルス分の情報だけが記録されることになります。それゆえ、それらを画像として可視化する場合、`full=True`と`full=False`の処理は同じになります。  
以下のセルはシミュレーションデータを変換するスクリプトです。


In [7]:
from src import mat2npz_sim
file_path_ex = "/home/smatsubara/documents/airlift/data/simulation/rawsignal/solid_liquid2.mat"
output_dir_ex = "/home/smatsubara/documents/airlift/data/simulation/processed"  # Change to your desired directory
config_path_ex="/home/smatsubara/documents/airlift/data/simulation/config.json"


mat2npz_sim(
    file_path=file_path_ex,
    config_path=config_path_ex,
    output_dir=output_dir_ex
)

<KeysViewHDF5 ['#refs#', '#subsystem#', 'kgrid', 'sensor_data']>
['Nt', 'Nx', 'Ny', 'Nz', 'dim', 'dt', 'dx', 'dxudxn', 'dxudxn_sgx', 'dy', 'dyudyn', 'dyudyn_sgy', 'dz', 'dzudzn', 'dzudzn_sgz', 'k', 'k_max', 'kx_max', 'kx_vec', 'ky_max', 'ky_vec', 'kz_max', 'kz_vec', 'nonuniform', 'xn_vec', 'xn_vec_sgx', 'yn_vec', 'yn_vec_sgy', 'zn_vec', 'zn_vec_sgz']
999999999.9999999
keys: ['#refs#', '#subsystem#', 'kgrid', 'sensor_data']
['#refs#', '#subsystem#', 'kgrid', 'sensor_data']
(100001,)
Processed data and metadata saved to: /home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz


'/home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz'

### 保存形式のチェック
実機・シミュレーションそれぞれにおいて、変換が正しく実行されたのかを確認します。変換されたファイルを読み込んで、それぞれのファイルの型を確認しましょう。以下はそのスクリプトです。

In [8]:
sim_file_processed="/home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz"
exp_file_processed="/home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz"

import numpy as np
with np.load(exp_file_processed) as data:
    #print(data.keys())
    print("======== experiment =========")
    print(data['original_keys'])
    print(f"(n_pulses, n_samples, n_channels): {data['processed_data'].shape}")
    print(data['fs'])


with np.load(sim_file_processed) as data:
    #print(data.keys())
    print("======== simulation =========")
    print(data['original_keys'])
    print(f"(n_pulses, n_samples, n_channels): {data['processed_data'].shape}")
    print(data['fs'])
    



======== experiment =========
['__header__' '__version__' '__globals__' 'Tstart' 'Tinterval'
 'ExtraSamples' 'RequestedLength' 'Length' 'Version' 'TDX1' 'TDX2' 'TDX3'
 'TDX1_enlarged']
(n_pulses, n_samples, n_channels): (14988, 5208, 4)
52083333.842615336
======== simulation =========
['#refs#' '#subsystem#' 'kgrid' 'sensor_data']
(n_pulses, n_samples, n_channels): (1, 100001, 1)
999999999.9999999


## 信号波形の可視化・解析
機械学習および信号波形の比較・解析をしやすいように変換した`.npz`ファイルを読み込み、可視化するスクリプトです。これら信号波形は画像形式に変換し解釈の精度を上げることもできます。一方で、一つのパルスの可視化にも意味があります。  
そこで、オプションに従いその画像化を行うスクリプトを`npz2png.py`という関数にまとめてあります。これらの引数として`full=True`ならば画像を、`full=False`ならば一パルス分の概形のみを返します。  
## npz2png (src/util.py)
### input 
 - file_path
 - save_path
 - channel_index
 - start_time
 - end_time
 - full(=True or False)
 - pulse_index  
### output
 - *_"pulse or img".png

In [9]:
from src.utils import npz2png
# npz2img関数を実際に使って画像を保存してみる例
npz_file_path = "/home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz"
npz_file_path_sim = "/home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz"
output_folder_path = "/home/smatsubara/documents/airlift/data/visualize"



npz2png(npz_file_path, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=True, pulse_index=0)
npz2png(npz_file_path, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=False, pulse_index=150)
#npz2png(npz_file_path, output_folder_path, channel_index=1, start_time=0.0, end_time=None, full=True, pulse_index=0)
npz2png(npz_file_path_sim, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=True, pulse_index=0)
npz2png(npz_file_path_sim, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=False, pulse_index=0)

(14988, 5208, 4)
device: cuda
/home/smatsubara/documents/airlift/data/visualize/P20241007-1013_processed_0img.png
(14988, 5208, 4)
/home/smatsubara/documents/airlift/data/visualize/P20241007-1013_processed_0pulse.png
(1, 100001, 1)
device: cuda
/home/smatsubara/documents/airlift/data/visualize/solid_liquid2_processed_0img.png
(1, 100001, 1)
/home/smatsubara/documents/airlift/data/visualize/solid_liquid2_processed_0pulse.png


### 補足
 - 処理に関して  
   生信号を画像化しても意味がないため、画像化の際はヒルベルト変換を実施
 - 前処理  
    飽和を起こしている領域の信号の値を０に（エラーハンドリングを目的とする。）
    データ転送は複数パルスの画像化の際のみ実行
 - 例外処理  
      シミュレーションにおいては０回目以外の測定の値を指定してもエラーを出力　　
      シミュレーション信号の画像生成に関しては、縦１ピクセルの画像と解釈
 - 備考  
      ０回目の測定結果の信号波形の可視化は、ややずれていることが多い